In [1]:
!pip install dash==2.9.3 jupyter-dash pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.2 MB/s eta 0:00:00


In [3]:
import pandas as pd

url = 'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
df = pd.read_csv(url)
df.head()


,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [4]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import plotly.express as px

# Sample chart (change for your dataset)
fig = px.scatter(df, x="gdpPercap", y="lifeExp", color="continent", hover_name="country", log_x=True)

# App setup
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Global Development Dashboard"),
    dcc.Graph(figure=fig)
])

# Run the app inline in Colab
app.run_server(mode='inline')


Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>

In [7]:
# prompt: make this dashboard more interactive by adding button,toggle etc

from dash.dependencies import Input, Output

# Sample chart (change for your dataset) - Keep this here or define it within the callback if it depends on inputs

# App setup
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Global Development Dashboard"),

    html.Label("Select Year:"),
    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    ),

    html.Label("Select Continent:"),
    dcc.Dropdown(
        id='continent-dropdown',
        options=[{'label': i, 'value': i} for i in df['continent'].unique()],
        value=df['continent'].unique()[0], # Default value
        multi=False
    ),

    html.Label("Select Country:"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': i, 'value': i} for i in df['country'].unique()],
        value=df['country'].unique()[0], # Default value
        multi=False
    ),


    html.Label("Toggle Log Scale for GDP:"),
    dcc.Checklist(
        id='log-gdp-toggle',
        options=[{'label': 'Use Log Scale', 'value': 'log'}],
        value=['log'] # Default value
    ),


    dcc.Graph(id='life-exp-vs-gdp')
])

# Define callback to update graph
@app.callback(
    Output('life-exp-vs-gdp', 'figure'),
    [Input('year-slider', 'value'),
     Input('continent-dropdown', 'value'),
     Input('country-dropdown', 'value'), # Add country-dropdown as an input
     Input('log-gdp-toggle', 'value')]
)
def update_graph(selected_year, selected_continent, selected_country, log_gdp_value):
    filtered_df = df[(df['year'] == selected_year) &
                     (df['continent'] == selected_continent) &
                     (df['country'] == selected_country)] # Filter by country

    log_x_status = 'log' in log_gdp_value

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", hover_name="country", # Removed color="country"
                     log_x=log_x_status, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig

# Add a callback to update the Country dropdown based on the selected continent
@app.callback(
    Output('country-dropdown', 'options'),
    [Input('continent-dropdown', 'value')]
)
def set_country_options(selected_continent):
    filtered_df = df[df['continent'] == selected_continent]
    return [{'label': i, 'value': i} for i in filtered_df['country'].unique()]

# Add a callback to update the default value of the Country dropdown
@app.callback(
    Output('country-dropdown', 'value'),
    [Input('country-dropdown', 'options')]
)
def set_country_value(available_options):
    if available_options:
        return available_options[0]['value']
    return None


# Run the app inline in Colab
if __name__ == '__main__':
    app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>